In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("HousePriceEDA") \
    .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/14 00:43:04 WARN Utils: Your hostname, soaz, resolves to a loopback address: 127.0.1.1; using 192.168.1.42 instead (on interface wlp0s20f3)
25/10/14 00:43:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/14 00:43:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Đọc dữ liệu

In [2]:
df = spark.read.csv('../data/socal2.csv', inferSchema= True, header= True)

In [3]:
df.show(5)
df.printSchema()


+--------+--------------------+---------------+------+---+----+----+------+
|image_id|              street|           citi|n_citi|bed|bath|sqft| price|
+--------+--------------------+---------------+------+---+----+----+------+
|       0|1317 Van Buren Av...|Salton City, CA|   317|  3| 2.0|1560|201900|
|       1|      124 C Street W|    Brawley, CA|    48|  3| 2.0| 713|228500|
|       2|     2304 Clark Road|   Imperial, CA|   152|  3| 1.0| 800|273950|
|       3|  755 Brawley Avenue|    Brawley, CA|    48|  3| 1.0|1082|350000|
|       4|2207 R Carrillo C...|   Calexico, CA|    55|  4| 3.0|2547|385100|
+--------+--------------------+---------------+------+---+----+----+------+
only showing top 5 rows
root
 |-- image_id: integer (nullable = true)
 |-- street: string (nullable = true)
 |-- citi: string (nullable = true)
 |-- n_citi: integer (nullable = true)
 |-- bed: integer (nullable = true)
 |-- bath: double (nullable = true)
 |-- sqft: integer (nullable = true)
 |-- price: integer (nul

In [4]:
print("Số hàng:", df.count())
print("Số cột:", len(df.columns))
df.describe().show()


Số hàng: 15474
Số cột: 8


25/10/14 00:43:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+----------------+------------+----------------+------------------+------------------+------------------+------------------+-----------------+
|summary|        image_id|      street|            citi|            n_citi|               bed|              bath|              sqft|            price|
+-------+----------------+------------+----------------+------------------+------------------+------------------+------------------+-----------------+
|  count|           15474|       15474|           15474|             15474|             15474|             15474|             15474|            15474|
|   mean|          7736.5|        NULL|            NULL|216.59751841799147|3.5063978286157425|2.4532506139328003|2173.9132092542327|703120.9375080781|
| stddev|4467.10336795557|        NULL|            NULL|112.37298461699699|1.0348377033393983|0.9587417665050789|1025.3396166979098|376976.1544210645|
|    min|               0|  0 H Street|    29 Palms, CA|                 0|                 1|

### Kiểm tra giá trị thiếu và trùng lặp

In [5]:
from pyspark.sql.functions import col, sum as _sum

# Đếm giá trị null mỗi cột
df.select([_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

# Đếm số dòng trùng
dup_count = df.count() - df.dropDuplicates().count()
print("Số dòng trùng lặp:", dup_count)


+--------+------+----+------+---+----+----+-----+
|image_id|street|citi|n_citi|bed|bath|sqft|price|
+--------+------+----+------+---+----+----+-----+
|       0|     0|   0|     0|  0|   0|   0|    0|
+--------+------+----+------+---+----+----+-----+

Số dòng trùng lặp: 0


### Phân tích cơ bản các cột

In [6]:
numeric_cols = ['n_citi', 'bed', 'bath', 'sqft', 'price']
df.select(numeric_cols).describe().show()


+-------+------------------+------------------+------------------+------------------+-----------------+
|summary|            n_citi|               bed|              bath|              sqft|            price|
+-------+------------------+------------------+------------------+------------------+-----------------+
|  count|             15474|             15474|             15474|             15474|            15474|
|   mean|216.59751841799147|3.5063978286157425|2.4532506139328003|2173.9132092542327|703120.9375080781|
| stddev|112.37298461699699|1.0348377033393983|0.9587417665050789|1025.3396166979098|376976.1544210645|
|    min|                 0|                 1|               0.0|               280|           195000|
|    max|               414|                12|              36.0|             17667|          2000000|
+-------+------------------+------------------+------------------+------------------+-----------------+



### Tần suất giá trị 

In [7]:
df.groupBy("citi").count().orderBy("count", ascending=False).show()


+--------------------+-----+
|                citi|count|
+--------------------+-----+
|       San Diego, CA|  707|
|     Los Angeles, CA|  388|
|       Lancaster, CA|  312|
|       La Quinta, CA|  305|
|       Riverside, CA|  274|
|          Corona, CA|  257|
|       Escondido, CA|  242|
|         Fontana, CA|  230|
|    Palm Springs, CA|  228|
|        Big Bear, CA|  228|
|        Palmdale, CA|  224|
|       Oceanside, CA|  208|
|Rancho Cucamonga, CA|  199|
|     Palm Desert, CA|  196|
|        Murrieta, CA|  188|
|     Paso Robles, CA|  184|
|  San Bernardino, CA|  177|
|  Lake Arrowhead, CA|  172|
|   Rancho Mirage, CA|  167|
|        Temecula, CA|  162|
+--------------------+-----+
only showing top 20 rows


### Hệ số tương phản giữa các biến số

In [8]:
for c in ['n_citi', 'bed', 'bath', 'sqft']:
    corr = df.stat.corr(c, 'price')
    print(f"Correlation({c}, price) = {corr}")


Correlation(n_citi, price) = 0.04716909254697919
Correlation(bed, price) = 0.3487107415125128
Correlation(bath, price) = 0.4777352482285269
Correlation(sqft, price) = 0.58345693239826


### Giá trung bình theo thành phố

In [10]:
df.groupBy("citi").avg("price").orderBy("avg(price)", ascending=False).show()


+--------------------+------------------+
|                citi|        avg(price)|
+--------------------+------------------+
|   Newport Coast, CA|         1819500.0|
|  Corona del Mar, CA|         1799995.0|
|         Solvang, CA|         1795000.0|
|Palos Verdes Esta...|         1784000.0|
|      Villa Park, CA|         1771999.8|
|     Westchester, CA|         1750000.0|
| Rancho Santa Fe, CA|1724987.5555555555|
|  West Hollywood, CA|         1679000.0|
|          Venice, CA| 1669142.857142857|
|        La Jolla, CA|        1653508.95|
|      El Segundo, CA|         1648640.0|
|      San Marino, CA|         1643250.0|
|   Playa del Rey, CA|         1622000.0|
|         Cardiff, CA|1616333.3333333333|
|  Valley Village, CA|         1593600.0|
|   Beverly Hills, CA|         1585750.0|
|         Del Mar, CA|1577181.8181818181|
|     Toluca Lake, CA|         1549000.0|
|   Hermosa Beach, CA|         1499000.0|
|   Newport Beach, CA|1487798.9523809524|
+--------------------+------------

### Giá trung bình theo số phòng ngủ

In [11]:
df.groupBy("bed").avg("price").orderBy("bed").show()


+---+------------------+
|bed|        avg(price)|
+---+------------------+
|  1|          496312.5|
|  2|490969.01347305387|
|  3|  636639.575828985|
|  4| 771009.4107390529|
|  5| 902858.4025122884|
|  6|1012255.6504297995|
|  7|1126643.1973684211|
|  8|1017620.5263157894|
|  9|1581462.6666666667|
| 10|1136583.3333333333|
| 11|         1550000.0|
| 12|          869500.0|
+---+------------------+

